In [1]:
# Import Library Dependencies
import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd
import numpy as np

# Import Custom API Functions
from apidata import quandl_stock_data

%matplotlib notebook

In [ ]:
# Two-Asset Portfolio


In [2]:
pfolio_assets = [
    "AMZN",
    "PFE",
    "AMD",
    "HAL",
    "PG",
    "COF",    
]


In [3]:
# Portfolio Stock Selection
pfolio_assets = [
    "NLY", 
    "T", 
    "AMZN", 
    "PG", 
    "BAC", 
    "PFE", 
    "FCX", 
    "MSFT", 
    "GM", 
    "QCOM",
]

print(f'Number of Portfolio Assets: {len(pfolio_assets)}')

Number of Portfolio Assets: 10


In [4]:
# Initializing DataFrame
portfolio = quandl_stock_data(pfolio_assets[0], verbose=True) \
            .rename(columns={"Close": pfolio_assets[0].upper()})[pfolio_assets[0].upper()] \
            .reset_index()


[Quandl] Query API Summary:

--------------------------------------------------------------------------- 

- symbol: NLY
- start_date: 2014-01-01
- end_date: 2019-01-01
- collapse: monthly
- data_type: pandas

 --------------------------------------------------------------------------- 


[Preview] Response DataFrame


              Open   High      Low  Close      Volume
Date                                                
2014-01-02   9.97  10.14   9.9700  10.01  10863700.0
2014-01-03  10.05  10.10   9.9200  10.00   9061700.0
2014-01-06  10.03  10.13  10.0300  10.11   9182200.0
2014-01-07  10.10  10.28  10.0900  10.25  15784100.0
2014-01-08  10.21  10.23  10.0600  10.15  11104000.0
2014-01-09  10.18  10.19   9.9701  10.08  11576700.0
2014-01-10  10.27  10.27  10.1000  10.24  16109100.0
2014-01-13  10.25  10.45  10.2000  10.39  15883400.0
2014-01-14  10.36  10.38  10.1700  10.19   8824300.0
2014-01-15  10.16  10.25  10.1400  10.14   6978500.0 

---------------------------------------

In [5]:
# Generate Portfolio DataFrame:
# [1] Query Stock Return Data - Quandl API Call
# [2] Merge Stock Returns into Portfolio DataFrame 

for i, stock in enumerate(pfolio_assets[1:]):
    print(f"<Quandl API Call> [{i+1}] Stock Symbol: {stock}")
    add_stock = quandl_stock_data(stock) \
                .rename(columns={"Close": stock})[stock] \
                .reset_index()
    
    portfolio = pd.merge(portfolio, add_stock, on="Date", how="inner")
    
print (f"\nAsset Returns: Merged // Portfolio DataFrame: <Complete>")

<Quandl API Call> [1] Stock Symbol: T
<Quandl API Call> [2] Stock Symbol: AMZN
<Quandl API Call> [3] Stock Symbol: PG
<Quandl API Call> [4] Stock Symbol: BAC
<Quandl API Call> [5] Stock Symbol: PFE
<Quandl API Call> [6] Stock Symbol: FCX
<Quandl API Call> [7] Stock Symbol: MSFT
<Quandl API Call> [8] Stock Symbol: GM
<Quandl API Call> [9] Stock Symbol: QCOM

Asset Returns: Merged // Portfolio DataFrame: <Complete>


In [6]:
# Set DataFrame Index (Date)
portfolio.set_index("Date", inplace=True)
portfolio.head(15)

,NLY,T,AMZN,PG,BAC,PFE,FCX,MSFT,GM,QCOM
Date,,,,,,,,,,
2014-01-02,10.01,34.95,397.97,80.54,16.10,30.46,37.63,37.160,40.95,73.32
2014-01-03,10.00,34.80,396.44,80.45,16.41,30.52,37.32,36.910,39.57,72.89
2014-01-06,10.11,34.96,393.63,80.64,16.66,30.55,37.02,36.130,40.40,72.70
2014-01-07,10.25,34.95,398.03,81.42,16.50,30.74,36.66,36.410,40.20,73.24
2014-01-08,10.15,34.24,401.92,80.24,16.58,30.95,36.20,35.760,40.42,73.68
2014-01-09,10.08,33.54,401.01,80.42,16.83,30.93,35.71,35.530,40.49,73.91
2014-01-10,10.24,33.62,397.66,80.30,16.77,30.69,36.17,36.040,40.03,73.87
2014-01-13,10.39,33.30,390.98,80.01,16.43,30.54,35.63,34.980,39.58,73.22
2014-01-14,10.19,33.48,397.54,80.87,16.77,31.00,36.12,35.780,40.02,73.39


In [ ]:
# View DataFrame -- Data Completeness
portfolio.count()

In [ ]:
# [Visualize] Plot Data Frame -- Closing Prices
visual_plot = (portfolio/portfolio.iloc[0] * 100).plot(figsize=(10,7))
plt.grid(axis="y", linestyle="dashed")

In [ ]:
# Convert Stock Price Data into Log Returns -- Removed Offset Row
log_returns = np.log(portfolio / portfolio.shift(1)).iloc[1:]
log_returns[:15]

In [ ]:
# Calculate Average Annual Log Returns 
log_returns.mean()*250

In [ ]:
# Portfolio Asset Covariance Matrix
log_returns.cov() * 250

In [ ]:
# Portfolio Asset Correlation Matrix
log_returns.corr()

In [ ]:
# Count Number of Assets in Portfolio
num_assets = (len(pfolio_assets) )
num_assets

In [ ]:
# Simulate 10k Random Portfolios
pfolio_sims = []
pfolio = {}

mkt_return = .098
sim_trials = 5000
for i in range(sim_trials):
    weights = np.random.random(num_assets)
    weights /= np.sum(weights)
    
    WTSp = zip(pfolio_assets, weights)
    RTNp = np.sum(weights * log_returns.mean()) * 250
    VOLp = np.sqrt(np.dot(weights.T, np.dot(log_returns.cov() * 250, weights)))
    
    pfolio = {a:round(wt, 4) for a,wt in WTSp}
    pfolio["RTN"] = round(RTNp, 4)
    pfolio["VOL"] = round(VOLp, 4)
    pfolio["Sharpe"] = round((RTNp - mkt_return)/VOLp, 4)
    pfolio_sims.append(pfolio)
    
    print("\n",pfolio, "\n")

print(">>> Data Points Generated -- Simulation Complete <<<")

In [ ]:
# Convert to DataFrame
portfolios = pd.DataFrame(pfolio_sims)
portfolios.head()

In [ ]:
# View Simulation Summary Statistics
portfolios[["Sharpe", "RTN", "VOL"]].describe()

In [ ]:
# Sort Simulated Portfolios: (Sharpe Ratio/Descending)
ranked_portfolios = portfolios.sort_values("Sharpe", ascending=False) \
                    .set_index(["Sharpe", "RTN", "VOL"])

In [ ]:
# View Best Portfolios (Most Efficient)
ranked_portfolios.head(10)

In [ ]:
# View Worst Portfolios (Least Efficient)
ranked_portfolios.tail(10)

In [ ]:
# Create DataFrames (Best Portfolios, Worst Portfolios, Remaining)
req_headers = ["Sharpe", "RTN", "VOL"]
efficient_pfolios = ranked_portfolios.reset_index()[req_headers].iloc[:50]
remaining_pfolios = ranked_portfolios.reset_index()[req_headers].iloc[950:]
all_pfolios = ranked_portfolios.reset_index()[req_headers]

In [ ]:
efficient_pfolios.plot(
    x="VOL",
    y="RTN",
    kind="scatter",
    color="red",
    figsize=(10,6)
)
plt.grid(True)
plt.title("Efficient Portfolio Composition")
plt.xlabel("Expected Portfolio Volatility")
plt.ylabel("Expected Portfolio Return")
plt.xlim(.05,.25)
plt.ylim(-.1, .25)


In [ ]:
remaining_pfolios.plot(
    x="VOL",
    y="RTN",
    kind="scatter",
    color="blue",
    alpha=.5,
    figsize=(10,6)
)
plt.grid(True)
plt.title("Ineffient Portfolio Composition")
plt.xlabel("Expected Portfolio Volatility")
plt.ylabel("Expected Portfolio Return")
plt.xlim(.05,.25)
plt.ylim(-.1, .25)

In [ ]:
all_pfolios.plot(
    x="VOL",
    y="RTN",
    kind="scatter",
    color="green",
    alpha=.5,
    figsize=(10,6)
)
plt.grid(True)
plt.title("Ineffient Portfolio Composition")
plt.xlabel("Expected Portfolio Volatility")
plt.ylabel("Expected Portfolio Return")
plt.xlim(.05,.25)
plt.ylim(-.1, .2)